In [27]:
import torch
import lightgbm as lgb
from tqdm import tqdm
from loaders.lending_loader import load_data, mono_list
from monotonenorm import SigmaNet, GroupSort
from sklearn.metrics import balanced_accuracy_score, accuracy_score
import numpy as np
import mup

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [63]:
Xtr, Ytr, Xts, Yts = load_data(get_categorical_info=False)
monotonic_constraints = [int(i in mono_list) for i in range(Xtr.shape[1])]

In [64]:
Xtr = Xtr[:30000]
Ytr = Ytr[:30000]
#Xts = Xts[:1000]
#Yts = Yts[:1000]

In [59]:
clf = lgb.LGBMRegressor(n_estimators=10000, max_depth=5, learning_rate=.1, monotone_constraint=monotonic_constraints)
clf.fit(Xtr, Ytr, early_stopping_rounds=200, eval_set=[(Xts, Yts)], eval_metric='mse', verbose=0)

LGBMRegressor(max_depth=5,
              monotone_constraint=[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              n_estimators=10000)

In [60]:
acc, bacc = 0,0
for i in np.linspace(0,1,50):
  acc = max(acc, accuracy_score(Yts, clf.predict(Xts)>i))
  bacc = max(bacc, balanced_accuracy_score(Yts, clf.predict(Xts)>i))
acc, bacc

(0.648450407337777, 0.649028777843047)

In [67]:

per_layer_lip = 2
width = 64

class Model(torch.nn.Module):
  def __init__(self, width, robust=False, sigma=False):
    super().__init__()
    if robust:
      from monotonenorm import direct_norm
      activation = lambda : GroupSort(2)
    else:
      direct_norm = lambda x, *args, **kwargs: x # make it a normal network
      activation = lambda : torch.nn.ReLU()

    self.nn = torch.nn.Sequential(
      direct_norm(torch.nn.Linear(Xtr.shape[1], width), kind="one-inf", alpha=per_layer_lip),
      activation(),
      direct_norm(torch.nn.Linear(width, width), kind="inf", alpha=per_layer_lip),
      activation(),
      direct_norm(torch.nn.Linear(width, width), kind="inf", alpha=per_layer_lip),
      activation(),
      direct_norm(mup.MuReadout(width, 1), kind="inf", alpha=per_layer_lip),
    )
    if sigma:
      self.nn = SigmaNet(self.nn, sigma=per_layer_lip**4, monotone_constraints=monotonic_constraints)
  
  def forward(self, x):
    return self.nn(x)

base = Model(1)
delta = Model(2)
model = Model(width)
mup.set_base_shapes(model, base, delta=delta)

for param in model.parameters():
    ### If initializing manually with fixed std or bounds,
    ### then replace with same function from mup.init
    # torch.nn.init.uniform_(param, -0.1, 0.1)
    mup.init.uniform_(param, -0.1, 0.1)
    ### Likewise, if using
    ###   `xavier_uniform_, xavier_normal_, kaiming_uniform_, kaiming_normal_`
    ### from `torch.nn.init`, replace with the same functions from `mup.init`


model = model.to(device)

optimizer = mup.MuAdam(model.parameters(), lr=1e-3)
print('params:', sum(p.numel() for p in model.parameters()))
#scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=.999)


params: 10241


In [68]:
print(model)
Xtrt = torch.tensor(Xtr, dtype=torch.float32).to(device)
Ytrt = torch.tensor(Ytr, dtype=torch.float32).view(-1, 1).to(device)
Xtst = torch.tensor(Xts, dtype=torch.float32).to(device)
Ytst = torch.tensor(Yts, dtype=torch.float32).view(-1, 1).to(device)

dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(Xtrt, Ytrt), batch_size=int(2**7), shuffle=True)

bar = tqdm(range(1000))
for i in bar:
  for Xi, yi in dataloader:
    y_pred = model(Xi)
    losstr = torch.nn.functional.binary_cross_entropy_with_logits(y_pred, yi)
    optimizer.zero_grad()
    losstr.backward()
    optimizer.step()
    #scheduler.step()

  with torch.no_grad():
    y_predts = model(Xtst)
    lossts = torch.nn.functional.binary_cross_entropy_with_logits(y_predts, Ytst)
    if i % 10 == 0:
      acc = 0
      for i in np.linspace(.3, .7, 50):
        acc = max(acc, accuracy_score(Ytst.cpu().numpy(), y_predts.cpu().numpy()>i))
    bar.set_description(f'Loss: {losstr.item():.4f} {lossts.item():.4f}, acc: {acc.item():.4f}')

Model(
  (nn): Sequential(
    (0): Linear(in_features=28, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
    (6): MuReadout(in_features=64, out_features=1, bias=True)
  )
)


Loss: 0.6385 0.6236, acc: 0.6402: 100%|██████████| 1000/1000 [07:42<00:00,  2.16it/s]
